# Report with IA

**Descrição**
* O objetivo do projeto foi criar um modelo de regressão com séries temporais, treinado com o intuito de prever o faturamento diário de uma cafeteria ao longo de um mês;
* O objetivo do deploy é criar um report com IA contendo as previsões do modelo. Dessa forma, o cliente poderia acompanhar bem as métricas e variações apresentadas de forma automatizada

**Etapas:**

* Prompt para a geração do relatório
* Configurações do relatório

# 1. Bibliotecas e importação dos dados

In [1]:
# Gerais

import pandas as pd
import matplotlib.pyplot as plt
import openai
from dotenv import load_dotenv
import os
import re

# Geração de PDF com o ReportLab

# Define o tamanho da página:
from reportlab.lib.pagesizes import letter
# Define os estilos de texto:
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
# Para a criação de documentos e elementos como parágrafos, tabelas e imagens:
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
# Para unidades de medida:
from reportlab.lib.units import inch
# Para usar cores:
from reportlab.lib import colors

# Bibliotecas para trabalhar com markdown e conversão (ex: HTML)

import markdown2

In [2]:
# Importação

df = pd.read_csv('data/report_data.csv',index_col=0)

In [3]:
df.head()

,ano-mes-dia,faturamento
0,2023-06-01,3501.3345
1,2023-06-02,3687.8699
2,2023-06-03,3512.5303
3,2023-06-04,3627.7790
4,2023-06-05,3570.6147


# 2. Acessando a API da OpenAI

In [4]:
# Acessando API

load_dotenv()
secret_key = os.getenv("OPEN_API_KEY")

In [5]:
# Instanciando

client = openai.OpenAI(api_key=secret_key)

# 3. Prompt para a geração do relatório com o GPT-4

In [6]:
# Passando o dataframe com as previsões para texto

data_text = df.to_string(index=False)
data_text

'ano-mes-dia  faturamento\n 2023-06-01    3501.3345\n 2023-06-02    3687.8699\n 2023-06-03    3512.5303\n 2023-06-04    3627.7790\n 2023-06-05    3570.6147\n 2023-06-06    3772.3198\n 2023-06-07    3763.6150\n 2023-06-08    3899.9277\n 2023-06-09    3988.8083\n 2023-06-10    4267.1177\n 2023-06-11    4279.8730\n 2023-06-12    4398.9785\n 2023-06-13    4417.1170\n 2023-06-14    4552.8600\n 2023-06-15    4664.5620\n 2023-06-16    4612.7760\n 2023-06-17    4499.0537\n 2023-06-18    4401.6606\n 2023-06-19    4496.7974\n 2023-06-20    4496.4673\n 2023-06-21    4352.6760\n 2023-06-22    4234.0557\n 2023-06-23    4042.9932\n 2023-06-24    4064.5315\n 2023-06-25    4172.6597\n 2023-06-26    4100.8960\n 2023-06-27    4026.1042\n 2023-06-28    3753.1160\n 2023-06-29    3528.1300\n 2023-06-30    3563.7275'

In [7]:
# Prompt gerado para o GPT-4

prompt = f"""
Você é um analista de dados. Baseado nas seguintes previsões de faturamento diário de uma cafeteria,
gere um relatório curto e conciso sumarizando as principais tendências, pontos notáveis e potenciais insights:

{data_text}

Seu relatório deve conter:
- Uma análise curta da tendência geral.
- Identificação de possíveis oscilações.
- Possíveis implicações de negócio.

Entregue uma análise concisa, porém informativa.
A análise deve ser devolvida em markdown.
Para fins de clareza, organize o relatório em seções bem definidas com títulos destacados (por exemplo, em negrito).
O restante do conteúdo pode seguir em texto simples.

"""

In [8]:
completion = client.chat.completions.create(
  model="gpt-4o-mini", store=True, temperature=0, max_tokens=1000,
  messages=[{"role": "system", "content": "Você é um analista de dados eficiente."},
              {"role": "user", "content": prompt}]
)

In [9]:
# Salvando a parte de interesse do output gerado pelo modelo

html_report = completion.to_dict()['choices'][0]['message']['content']
html_report

'# Relatório de Análise de Faturamento Diário da Cafeteria - Junho de 2023\n\n**Tendência Geral**\n\nO faturamento diário da cafeteria apresentou uma tendência de crescimento ao longo do mês de junho de 2023. O valor inicial de R$ 3.501,33 no dia 1º subiu para R$ 3.563,73 no dia 30, com um pico de R$ 4.398,98 no dia 12. A média de faturamento diário ao longo do mês foi de aproximadamente R$ 4.000,00, indicando um desempenho positivo e um aumento na demanda.\n\n**Oscilações Notáveis**\n\nApesar da tendência de crescimento, foram observadas algumas oscilações significativas. O faturamento atingiu seu ponto mais alto no dia 12, seguido por uma leve queda nos dias subsequentes, com valores variando entre R$ 4.000,00 e R$ 4.600,00 até o dia 18. A partir do dia 19, o faturamento começou a apresentar uma tendência de queda, com valores caindo para cerca de R$ 3.500,00 até o final do mês. Essa oscilação pode ser atribuída a fatores como variações na demanda, promoções ou eventos locais.\n\n**I

In [10]:
# html_report = '### Relatório de Análise de Faturamento Diário da Cafeteria - Junho de 2023\n\n#### Tendência Geral\nO faturamento diário da cafeteria apresentou uma tendência de crescimento ao longo do mês de junho, com um aumento significativo nos primeiros 15 dias. O faturamento subiu de R$ 3.501,33 no dia 1º para R$ 4.664,56 no dia 15, representando um crescimento de aproximadamente 33%. Após esse pico, observou-se uma leve queda, estabilizando-se em torno de R$ 4.000,00 nos últimos dias do mês.\n\n#### Oscilações Notáveis\n- **Pico de Faturamento**: O maior faturamento foi registrado no dia 15, com R$ 4.664,56.\n- **Queda e Estabilização**: Após o dia 15, o faturamento começou a oscilar, com uma queda acentuada nos dias 28 e 29, onde o faturamento caiu para R$ 3.753,12 e R$ 3.528,13, respectivamente. Essa oscilação sugere uma possível sazonalidade ou impacto de fatores externos, como clima ou eventos locais.\n\n#### Implicações de Negócio\n1. **Promoções e Marketing**: O aumento do faturamento no início do mês pode indicar uma resposta positiva a campanhas de marketing ou promoções. É recomendável analisar quais ações foram implementadas e considerar a repetição ou ampliação dessas estratégias.\n   \n2. **Gestão de Estoque**: A oscilação no faturamento sugere a necessidade de uma gestão de estoque mais dinâmica, especialmente em relação a produtos que podem ter alta demanda em dias específicos.\n\n3. **Análise de Sazonalidade**: A queda no final do mês pode indicar uma sazonalidade que deve ser investigada. Compreender os padrões de consumo pode ajudar a planejar melhor as operações e maximizar o faturamento.\n\n4. **Feedback do Cliente**: Considerar a coleta de feedback dos clientes durante os dias de menor faturamento pode fornecer insights sobre possíveis melhorias no serviço ou no cardápio.\n\nEm resumo, o mês de junho apresentou um desempenho positivo, mas a análise das oscilações e a implementação de estratégias baseadas em dados podem ajudar a otimizar ainda mais o desempenho da cafeteria.'

# 4. Configurações do relatório

### 4.1 Configurações do gráfico

In [11]:
# Definição do caminho do gráfico
graph_path = "data/grafico.png"

# Criando o gráfico de tendência de faturamento
plt.figure(figsize=(20, 10), dpi=300)
plt.plot(df['ano-mes-dia'], df['faturamento'], marker='s', linestyle='-', color='#D2691E')
plt.title("Tendência de faturamento", fontsize=14)
plt.xlabel("Data", fontsize=12)
plt.ylabel("Faturamento (R$)", fontsize=12)

# Adicionando os valores nos pontos do gráfico
for i, valor in enumerate(df['faturamento']):
    plt.text(df['ano-mes-dia'][i], valor, f'{valor:.2f}', fontsize=7, ha='center', va='bottom')

plt.xticks(rotation=45)
plt.savefig(graph_path)
plt.close()

### 4.2 Configurações do pdf

In [12]:
# Criando o relatório em PDF
html_report = markdown2.markdown(html_report)

pdf_filename = "sales_report.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter, leftMargin=40, rightMargin=40, topMargin=20, bottomMargin=20)
styles = getSampleStyleSheet()
story = []

# Estilos personalizados
styles["Title"].fontSize = 18
styles["Title"].spaceAfter = 12
styles["Title"].textColor = "#8B4513"
styles["Heading1"].fontSize = 14
styles["Heading1"].spaceBefore = 12
styles["Heading1"].spaceAfter = 6
styles["Heading1"].textColor = colors.black
styles["BodyText"].fontSize = 12
styles["BodyText"].leading = 14
styles["BodyText"].spaceAfter = 6

# Adicionando o título
title = Paragraph("<b>Relatório de Previsão de Faturamento</b>", styles["Title"])
story.append(title)
story.append(Spacer(1, 0.2 * inch))

# Configurações do parágrafo
for paragraph in html_report.split("\n"):
    if paragraph.strip():
        story.append(Paragraph(paragraph, styles["BodyText"]))
        story.append(Spacer(1, 0.2 * inch))

### 4.3 Configurações do pdf

In [13]:
# Criando a tabela
data_table = [["Data", "Previsão (R$)"]] + df[['ano-mes-dia', 'faturamento']].values.tolist()
table = Table(data_table, colWidths=[150, 150])
table.setStyle(TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), '#8B4513'),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 8),
    ('BACKGROUND', (0, 1), (-1, -1), '#F5F5DC'),
    ('GRID', (0, 0), (-1, -1), 1, '#D3D3D3'),
]))

story.append(Spacer(1, 0.3 * inch))
story.append(Paragraph("Previsões de faturamento:", styles["Heading1"]))
story.append(Spacer(1, 0.1 * inch))
story.append(table)

# Adicionando o gráfico
story.append(Paragraph("Tendência de faturamento", styles["Heading1"]))
story.append(Spacer(1, 0.2 * inch))
story.append(Image(graph_path, width=700, height=400))
story.append(Spacer(1, 0.5 * inch))

In [14]:
# Gerando o PDF
doc.build(story)

# 5. Automatizando a geração do relatório

In [15]:
def generate_coffee_forecast_report(df, model='gpt-4o-mini', max_tokens=1000, temperature=0):
    """
    Gera um relatório conciso baseado em previsões de faturamento diário de uma cafeteria.

    Parâmetros:
    - df: DataFrame com os dados de previsão.
    - model: Modelo da OpenAI a ser usado.
    - max_tokens: Número máximo de tokens a serem gerados.
    - temperature: Temperatura da geração (controle de criatividade).

    Retorna:
    - Um relatório textual com tendências, oscilações e implicações de negócio.
    """
    
    # Convertendo o DataFrame para texto
    data_text = df.to_string(index=False)
    
    # Criando o prompt
    prompt = f"""
    Você é um analista de dados. Baseado nas seguintes previsões de faturamento diário de uma cafeteria, gere um relatório curto e conciso sumarizando as principais tendências, pontos notáveis e potenciais insights:

    {data_text}

    Seu relatório deve conter:
    - Uma análise curta da tendência geral.
    - Identificação de possíveis oscilações.
    - Possíveis implicações de negócio.

    Entregue uma análise concisa, porém informativa.
    A análise deve ser devolvida em markdown.
    """

    # Chamando o modelo da OpenAI
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=[
            {"role": "system", "content": "Você é um analista de dados eficiente."},
            {"role": "user", "content": prompt}
        ]
    )

    # Extraindo o conteúdo do relatório
    report = completion.to_dict()['choices'][0]['message']['content']
    
    return report